In [1]:
import os
import requests
import json
import datetime
import time
import pandas as pd
from pprint import pprint


In [ ]:
#function to convert a list of dictionaries into a simplified list of dictionaries. 
#This function can be used to convert a list, generated from a complete year returns of billboard weekly 100 queries.
#function returns a list of dictionaries. Each dictionary describes ones track, and includes datetime, title, artist and year.s


def convert_year_list_to_simple_list(x):

    song_list = []
    
    for i, k in enumerate(x):
        date = list(k.keys())[0]
    
        date_str = f'{date}'
        date_format = '%Y-%m-%d'
        date_obj_year = datetime.datetime.strptime(date_str, date_format).year
    
        content = k[date]
    
        for n in content.values():
    
            track = {}
    
            title = n["title"]
            artist = n["artist"]
    
            track["date"] = date
            track["title"] = title
            track["artist"] = artist
            track["year"] = date_obj_year
    
            song_list.append(track)

    return song_list
        


In [ ]:
#this is the first iteration of this function. it has been since modified to address project needs
#consolidate_top100() takes in the output of convert_year_list_to_simple_list(), and returns a DataFrame with the 100 most recurring songs.

def consolidate_top100_original(x):

    #converting raw list of dictionaries into a pandas dataframe.
    # this requires a list with a specific formatting, generated by convert_year_list_to_simple_list()
    df1 = pd.DataFrame(x)

    #remove data column. Rows with the same song title and artist are made unique by date.
    df2 = df1.drop("date" ,axis=1)

    #using the pandas pivot_table() function to get a count of song titles
    #the pivot table is converted into a new DataFrame.
    df3 = pd.DataFrame(df2.pivot_table(index=["title"], aggfunc='size'))

    #create a new DataFrame by merging df2 and df3, to combine track info with the associated count.
    #remove lines with NaN values
    #remove duplicate rows (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html)
    #https://pandas.pydata.org/docs/user_guide/merging.html
    df4 = pd.merge(df3, df2, how="left", on="title").dropna().drop_duplicates()

    #sorting the DataFrame by count in descending order (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html)
    #reordering columns for csv readability
    df4 = df4.sort_values(by=[0], ascending=False)
    df4 = df4.iloc[:,[3,0,2,1]]
    df4 = df4.rename(columns={0:"count"}) #https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html

    #creating a new DataFrame that is 100 songs with the greatest count value
    # https://stackoverflow.com/questions/12021754/how-to-slice-a-pandas-dataframe-by-position
    final_df = df4.head(100)

    #getting 'year' value from DataFrame to apply a distinguishing element to the outputted .csv
    #https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html#pandas.DataFrame.to_csv
    year = str(final_df.iloc[0,0])
    path = r"C:\Users\kevin\bootcamp\Project_1\resources"
    final_df.to_csv(os.path.join(path,f"top100_{year}.csv")) #https://stackoverflow.com/questions/22872952/set-file-path-for-to-csv-in-pandas

    return final_df

In [ ]:
#create list of dates to pass to Billboard API (https://stackoverflow.com/questions/993358/creating-a-range-of-dates-in-python)
base = datetime.date.today()
#getting one date per week (https://docs.python.org/3/library/datetime.html#datetime.date)
date_list = [base - datetime.timedelta(days=(x*7)) for x in range(2340)]


year_2024 = date_list[0:44]
#can extract year value from datetime.date object by using the .year attribute



In [ ]:
#maaking calls to the Billboard Weekly Hot 100 endpoint
billboard_url = "https://billboard-api2.p.rapidapi.com/hot-100"
headers = {
	"x-rapidapi-key": "65155348c9mshaa21262afddd264p14c37bjsn21ec97b632c2",
	"x-rapidapi-host": "billboard-api2.p.rapidapi.com"
}

list_2024 = []


for date in year_2024:
    
    try:
        querystring = {"date":f"{date}","range":"1-100"}
        
        response = requests.get(billboard_url, headers=headers, params=querystring).json()
        content = response["content"]
        week_data = {}
        week_data[f"{date}"] = content
        list_2024.append(week_data)
        time.sleep(0.25)
        print(f"success on {date}")
    except:
        print(f"error on call for date: {date}")
        pass


In [ ]:
topTracks_2024 = convert_year_list_to_simple_list(list_2024)
pprint(topTracks_2024)
#print(len(topTracks_2024))


In [ ]:
top100_2024 = consolidate_top100(topTracks_2024)

In [ ]:
top100_2024.head(10)

In [ ]:
#create list of dates to pass to Billboard API (https://stackoverflow.com/questions/993358/creating-a-range-of-dates-in-python)
base = datetime.date.today()
#getting one date per week (https://docs.python.org/3/library/datetime.html#datetime.date)
date_list = [base - datetime.timedelta(days=(x*7)) for x in range(2340)]

#can extract year value from datetime.date object by using the .year attribute
#year_2024 = date_list[0:44]

all_years_list = []
dates_in_year = []

current_year = 2024


test_date1 = date_list[0]
test_date2 = date_list[100]
test_date3 = date_list[300]

new_test_list = [test_date1, test_date2, test_date3]



#for date in date_list:
for date in new_test_list:

    year_value = int(date.year)

    if year_value == current_year:
        dates_in_year.append(date)
    elif year_value != current_year:
        all_years_list.append(dates_in_year)
        dates_in_year = []
        current_year = current_year -1


print(all_years_list)

"""
count = 0
for i in all_years_list:
    #print(len(i))
    for x in i:
        count += 1
#pprint(all_years_list)

    
print(f"the number of dates contained in all_years_list = {count}")  
print(f"the number of dates in original date_list = {len(date_list)}")
"""




    


In [10]:
df20 = pd.read_csv(r"C:\Users\kevin\bootcamp\Project_1\resources\billboard_hot_100_csvs\2024_bb_hot_100.csv")
df20.head()

,rank,title,artist,image,weeks at no.1,last week,peak position,weeks on chart,detail,date
0,1,Rockin' Around The Christmas Tree,Brenda Lee,https://charts-static.billboard.com/img/1960/1...,1.0,2.0,1,58,up,2024-01-06
1,2,All I Want For Christmas Is You,Mariah Carey,https://charts-static.billboard.com/img/1994/1...,NaN,1.0,1,65,down,2024-01-06
2,3,Jingle Bell Rock,Bobby Helms,https://charts-static.billboard.com/img/1958/1...,NaN,3.0,3,56,same,2024-01-06
3,4,Last Christmas,Wham!,https://charts-static.billboard.com/img/1998/0...,NaN,4.0,4,38,same,2024-01-06
4,5,A Holly Jolly Christmas,Burl Ives,https://charts-static.billboard.com/img/1998/0...,NaN,5.0,4,39,same,2024-01-06


In [20]:
#consolidate_top100() takes in a csv file containing all songs from one year, and returns a DataFrame with the 100 most recurring songs.

def consolidate_top100_fromcsv_add_year(x):

    #converting raw list of dictionaries into a pandas dataframe.
    # this requires a list with a specific formatting, generated by convert_year_list_to_simple_list()
    #df1 = pd.DataFrame(x)
    df1 = x
    #remove data column. Rows with the same song title and artist are made unique by date.

    date_list = df1["date"].to_list()
    new_date_list = []
    for i in date_list:
        date_format = '%Y-%m-%d'
        year_value = datetime.datetime.strptime(i, date_format).year
        new_date_list.append(year_value)


    df2 = df1.assign(year=new_date_list)
    
    df2 = df2.drop(["date","rank","image","weeks at no.1","last week", "peak position", "weeks on chart", "detail"] ,axis=1)
    

    #using the pandas pivot_table() function to get a count of song titles
    #the pivot table is converted into a new DataFrame.
    df3 = pd.DataFrame(df2.pivot_table(index=["title"], aggfunc='size'))

    #create a new DataFrame by merging df2 and df3, to combine track info with the associated count.
    #remove lines with NaN values
    #remove duplicate rows (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html)
    #https://pandas.pydata.org/docs/user_guide/merging.html
    df4 = pd.merge(df3, df2, how="left", on="title").dropna().drop_duplicates(subset=["title"])#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html

    #sorting the DataFrame by count in descending order (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html)
    #reordering columns for csv readability
    df4 = df4.sort_values(by=[0], ascending=False)
    #df4 = df4.iloc[:,[3,0,2,1]]
    df4 = df4.rename(columns={0:"count"}) #https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html
    df4 = df4.loc[:,["title","artist","count","year"]]

    #creating a new DataFrame that is 100 songs with the greatest count value
    # https://stackoverflow.com/questions/12021754/how-to-slice-a-pandas-dataframe-by-position
    final_df = df4.head(100).reset_index(drop=True)#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html

    #getting 'year' value from DataFrame to apply a distinguishing element to the outputted .csv
    #https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html#pandas.DataFrame.to_csv
    #year = str(final_df.iloc[0,0])
    path = r"C:\Users\kevin\bootcamp\Project_1\resources\most_frequent_hot100songs_by_year"
    #final_df.to_csv(os.path.join(path,f"top100_{year}.csv")) #https://stackoverflow.com/questions/22872952/set-file-path-for-to-csv-in-pandas
    final_df.to_csv(os.path.join(path,f"top100_{new_date_list[0]}.csv")) #https://stackoverflow.com/questions/22872952/set-file-path-for-to-csv-in-pandas

    return final_df




In [18]:
consolidate_top100_fromcsv_add_year(df20)

,title,artist,count,year
0,Lose Control,Teddy Swims,43,2024
1,Stick Season,Noah Kahan,40,2024
2,Beautiful Things,Benson Boone,39,2024
3,Houdini,Dua Lipa,37,2024
4,I Remember Everything,Zach Bryan Featuring Kacey Musgraves,35,2024
...,...,...,...,...
95,Euphoria,Kendrick Lamar,16,2024
96,Snooze,SZA,16,2024
97,Burn It Down,Parker McCollum,16,2024
98,Casual,Chappell Roan,16,2024


In [29]:
year_list = [2024-x for x in range(0,45)]


45


In [22]:
#passing all raw hot 100 csv's into function, generating top 100 lists per csv and dumping into new csv


for year in year_list:
    path = f"C:\\Users\\kevin\\bootcamp\\Project_1\\resources\\billboard_hot_100_csvs\\{year}_bb_hot_100.csv"
    df = pd.read_csv(path)
    consolidate_top100_fromcsv_add_year(df)

In [28]:
#reading in all top100 csvs, and concatenating into a single dataframe. dumping new consolidating dataframe into csv file.


df_csv_append = pd.DataFrame()

for year in year_list:
    path = f"C:\\Users\\kevin\\bootcamp\\Project_1\\resources\\most_frequent_hot100songs_by_year\\top100_{year}.csv"
    df = pd.read_csv(path)
    df_csv_append = pd.concat([df_csv_append, df], ignore_index=True)




final_path = r"C:\Users\kevin\bootcamp\Project_1\resources"
df_csv_append.to_csv(os.path.join(final_path,f"all_top100songs_by_year.csv")) #https://stackoverflow.com/questions/22872952/set-file-path-for-to-csv-in-pandas
